<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:10px 5px'> 
Master Thesis Yannik Haller - Data Preprocessing VADER
</h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1. Load required packages and the data
</h2>
</div>

In [1]:
# Import required baseline packages
import re
import os
import glob
import time
import sys
import pandas as pd
import numpy as np
from pprint import pprint

# Change pandas' setting to print out long strings
pd.options.display.max_colwidth = 200

# Spacy (for lemmatization)
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
# Set the appropriate working directory
os.chdir('D:\\Dropbox\\MA_data')

In [3]:
# Read in the aggregated data
fr_tx = pd.read_csv("agg_csv_sparse_fr.csv", index_col = 0, dtype = {'so': object, 'la': object, 'tx': object})

In [4]:
# Take a look at the shape of the data
fr_tx.shape

(481162, 3)

In [5]:
# Store the article IDs (i.e. index)
fr_idx = fr_tx.index  # French

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2. Preprocess the text data
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.1 Define all required functions to preprocess the data
</h2>
</div>

In [6]:
# Define a function to prepare/pre-clean the text data for the sentiment analysis using the VADER classifier
def pre_clean_vader(articles):
    # Raise an error if an inappropriate data type is given as an input
    if(not isinstance(articles, list)):
        raise ValueError("Invalid input type. Expected a list.")

    # Keep track of the processing time
    t = time.time()

    # Remove any links starting with http:// or https://
    articles = [re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]
    # Remove any links starting with www.
    articles = [re.compile('www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]

    # Replace punctuations which are not followed by a blank with punctuations followed by a blank
    articles = [re.sub(r'[\.]', '. ', x) for x in articles]
    # Separate words in which a lowercase letter is followed by a capital letter, since they usually do not belong together
    articles = [re.sub('(^[a-z]*)+([A-Z])', r'\1 \2', x) for x in articles]
    # Correct manually for those cases where a name like 'McDonalds' was separated to Mc Donalds
    articles = [re.sub('Mc ', 'Mc', x) for x in articles]
    # Replace quotation marks with a blank
    articles = [re.sub('«', ' ', x) for x in articles]
    articles = [re.sub('»', ' ', x) for x in articles]
    # Remove percentage signs
    articles = [re.sub('%', ' ', x) for x in articles]
    # Remove distracting hyphens
    articles = [re.sub("-", " ", x) for x in articles]
    articles = [re.sub("–", " ", x) for x in articles]
    # Remove any blank that precedes a comma
    articles = [re.sub(" ,", ",", x) for x in articles]
    # Remove any blank that precedes a dot
    articles = [re.sub(" \.", ".", x) for x in articles]
    
    # Replace control characters (e.g. \n or \t) and multiple blanks with a single blank
    articles = [re.sub('\s+', ' ', x) for x in articles]
    # Print out the processing time
    print("Processing time for pre-cleaning: ", str((time.time() - t)/60), "minutes")

    # Return the pre-cleaned text data
    return articles

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.2 Apply the preprocessing
</h2>
</div>

In [7]:
# Apply the preprocessing by means of the previously defined function
fr_tx = pre_clean_vader(fr_tx.tx.values.tolist())

Processing time for pre-cleaning:  2.8476298451423645 minutes


In [8]:
# Take a look at the first element of the preprocessed data
fr_tx[0]

" La Bourse de New York a terminé en hausse mercredi, sur les espoirs d'un prochain accord sur un nouveau plan d'aide économique américain qui a mené le Dow Jones brièvement au dessus de 2 en séance. Le Dow Jones Industrial Average a avancé de 1,20 à 27. 781,70 points. Le Nasdaq a gagné 0,74 à 11. 167,50 points et le S&P 500, a progressé de 1,05 à 3370,53 points. La Bourse de New York avait clôturé anxieusement en légère baisse mardi avant le débat présidentiel. Le Dow Jones Industrial Average, avait cédé 0,48 et le Nasdaq 0,29. Mercredi, la rencontre entre la cheffe des démocrates à la Chambre et le secrétaire américain au Trésor pour discuter d'une nouvelle aide économique, en panne depuis des mois, a suscité l'espoir d'un compromis raisonnable, selon les mots de Steven Mnuchin. Cet optimisme a donné un coup de fouet aux actions, qui s'est brusquement tempéré lorsque le chef des républicains au Sénat Mitch McConnell est sorti et a dit que les positions étaient encore très, très éloig

In [9]:
# Compare the fully preprocessed data to the initial text

"La Bourse de New York a terminé en hausse mercredi, sur les espoirs d'un prochain accord sur un nouveau plan d'aide économique américain qui a mené le Dow Jones brièvement au-dessus de 2% en séance.Le Dow Jones Industrial Average a avancé de 1,20% à 27.781,70 points. Le Nasdaq a gagné 0,74% à 11.167,50 points et le S&P 500, a progressé de 1,05% à 3370,53 points.La Bourse de New York avait clôturé anxieusement en légère baisse mardi avant le débat présidentiel. Le Dow Jones Industrial Average, avait cédé 0,48% et le Nasdaq -0,29%.Mercredi, la rencontre entre la cheffe des démocrates à la Chambre et le secrétaire américain au Trésor pour discuter d'une nouvelle aide économique, en panne depuis des mois, a suscité l'espoir d'un «compromis raisonnable», selon les mots de Steven Mnuchin.Cet optimisme a donné un coup de fouet aux actions, qui s'est brusquement tempéré «lorsque le chef des républicains au Sénat Mitch McConnell est sorti et a dit que les positions étaient encore très, très éloignées», a expliqué Karl Haeling de LBBW.La Bourse de New York a vu aussi l'introduction en fanfare, via une cotation directe, des titres du discret groupe de surveillance de données Palantir, à un prix le valorisant à plus de 21 milliards de dollars. Sous le symbole PLTR, le titre a clôturé à 9,73 dollars, soit bien au dessus du prix indicatif de 7,25 dollars donné mardi soir par le New York Stock Exchange (lire page 11)Le titre du fabricant de camions électriques et à hydrogène Nikola a repris de la vigueur (+14,54% à 20,48 dollars) après sa descente aux enfers marquée par la perte de deux tiers de sa valeur depuis son introduction en bourse.Nikola a ajourné mercredi un événement au cours duquel il devait présenter en grande pompe son nouveau pick-up Badger.Quasiment tous les secteurs du S&P ont terminé dans le vert, celui de la santé en tête. les laboratoires Pfizer ont pris 1,41%. Des grands noms de la tech ont progressé comme Microsoft et Apple (+1,50%).Sur le marché obligataire, le taux à 10 ans sur la dette américaine augmentait à 0,6840% contre 0,6495% mardi soir. – (afp)"

In [10]:
# Create a correctly indexed dataframe containing the preprocessed fulltext data in a column and export it as a csv file
pd.DataFrame(fr_tx, index = fr_idx, columns = ['tx']).to_csv("Preprocessed/Sentiment_Analysis/Fulltext/fr_fulltext_senti.csv", index = True, encoding = 'utf-8-sig')
pd.DataFrame(fr_tx, index = fr_idx, columns = ['tx']).to_csv("Preprocessed/Fulltext/fr_fulltext.csv", index = True, encoding = 'utf-8-sig')

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
3. Read in and inspect the preprocessed fulltext data
</h2>
</div>

In [11]:
# Define a function to read in the preprocessed fulltext data
def read_preprocessed_fulltext(language):
    # Raise an error if an inadmissible language is chosen
    allowed_languages = ['de', 'en', 'fr', 'it']
    if language not in allowed_languages:
        raise ValueError("Invalid language. Expected one of: %s" % allowed_languages)
    
    # Set the appropriate working directory
    os.chdir('D:\\Dropbox\\MA_data')

    # Define the name of the file to load
    filename = "Preprocessed/Sentiment_Analysis/Fulltext/"+language+"_fulltext_senti.csv"

    # Read in the data
    tx_ft = pd.read_csv(filename, index_col = 0, dtype = {'tx': object})

    # Get the articles' index together with an enumeration to identify their position in the list of precleaned articles
    idx = tx_ft.index
    idx = pd.DataFrame(idx, columns = [language+'_idx'])

    # Extract the text data as a list of articles
    tx_ft = tx_ft.tx.to_list()

    # Return the preprocessed data
    return tx_ft, idx

In [12]:
# Read in the preprocessed fulltext data
fr_tx_ft, fr_idx = read_preprocessed_fulltext('fr')

In [13]:
# Take a look at the first element of the preprocessed fulltext data
fr_tx_ft[0]

" La Bourse de New York a terminé en hausse mercredi, sur les espoirs d'un prochain accord sur un nouveau plan d'aide économique américain qui a mené le Dow Jones brièvement au dessus de 2 en séance. Le Dow Jones Industrial Average a avancé de 1,20 à 27. 781,70 points. Le Nasdaq a gagné 0,74 à 11. 167,50 points et le S&P 500, a progressé de 1,05 à 3370,53 points. La Bourse de New York avait clôturé anxieusement en légère baisse mardi avant le débat présidentiel. Le Dow Jones Industrial Average, avait cédé 0,48 et le Nasdaq 0,29. Mercredi, la rencontre entre la cheffe des démocrates à la Chambre et le secrétaire américain au Trésor pour discuter d'une nouvelle aide économique, en panne depuis des mois, a suscité l'espoir d'un compromis raisonnable, selon les mots de Steven Mnuchin. Cet optimisme a donné un coup de fouet aux actions, qui s'est brusquement tempéré lorsque le chef des républicains au Sénat Mitch McConnell est sorti et a dit que les positions étaient encore très, très éloig

In [14]:
# Take a look at the dataframe containing the according index
fr_idx

,fr_idx
0,0
1,1
2,2
3,3
4,4
...,...
481157,2436478
481158,2436479
481159,2436480
481160,2436481


In [15]:
# Remove unnecessary variables to save RAM
del fr_tx_ft, fr_idx